In [5]:
import random
import pandas as pd

df_ademe_mt = pd.read_csv("../data/ademe_mt_75.csv")

df_ademe_mt.columns

columns = ['dateConvention_ademe', 'nomBeneficiaire_ademe',
       'montant_ademe', 'pourcentageSubvention_ademe', 'DEPET_ademe', 'naf1etlib_ademe',
       'naf2etlib_ademe', 'naf3etlib_ademe', 'naf4etlib_ademe',
       'naf5etlib_ademe','id_mt',  'environmentalTopics_mt', 'fundingTypes_mt',
        'subventionRateUpperBound_mt',
       'subventionRateLowerBound_mt',
       'applicationStartDate_mt']

df = df_ademe_mt[columns]

## Cas négatifs

   Les données contenues dans le tableau df représentes des bonnes association au sens où chaque ligne représente une entreprise qui a postulé pour une aide et qui l'a obtenue. Nous appelerons positives une telle donnée. Nous aimerions avoir dans l'idéal des données de couple (aide, entreprise) qui ne fonctionnent pas, que nous appelerons négatives. Pour synthétyser ces données nous nous proposons de faire l'hypothese suivant :

Supposons ici qu'il y a réciprocité entre la volonté/capacité à obtenir une aide et l'existence dans les nos données df. Autrement dit si un entreprise avait voulu/pu obtenir une aide ce couple serai présent dans nos données. Ainsi, tout les couples (aide,entreprise) qui ne sont pas dans le tableau df sont considérés comme négatifs. Evidement cette hypothèse n'est pas idéale, mais elle permet de placer, modulo le choix de ces cas négatifs dans le cadre d'un simple problème de classification.

On note avec $\mathcal{A}$ l'ensemble des aides présentes dans le dataset df, et $\mathcal{C}$ l'ensemble des entreprises présentes dans le dataset df. Enfin on note $\mathcal{D^+}  \subset \mathcal{A} \times \mathcal{C}$, les données positives de df, c'est-à-dire les paires $(a, c)$ qui sont dans df.

On veut ajouter à notre dataset un ensemble $\mathcal{D}^-  \subset \mathcal{A} \times \mathcal{C}$ de couples négatifs, au sens où ils n'appartiennent pas déjà à $\mathcal{D}^+$. On alors un très grand nombre de candidats. Si on note $\mathcal{N}$ l'ensemble des paires de couples (aide, entreprise) négatifs au sens de la supposition ci-dessus, on obtient :

$\mathcal{N} =  \mathcal{A} \times \mathcal{C} - \mathcal{D}^+$.

ici, $|\mathcal{A}| = 27$, $|\mathcal{C}| = 450$ et $|\mathcal{P}| = 474$

D'où, $|\mathcal{N}| = |\mathcal{A}| \times |\mathcal{C}| -  |\mathcal{P}| = 27 \times 450 - 474  = 11676$.

On ne veut pas ajouter à nos données tout les couples de $\mathcal{N}$, au risque de désiquilibrer notre jeu de données. De même, on aimerai que le sous ensemble $\mathcal{D}^- \subset \mathcal{N}$ des couples négatifs à ajouter à notre dataset respecte les conditions suivantes :
- $|\mathcal{D}^-| = |\mathcal{D}^+|$
- $\forall c \in \mathcal{C},\ |\{(a,c) \in \mathcal{D}^-\}| = |\{(a,c) \in \mathcal{D}^+\}|$ 
- $\forall a \in \mathcal{A},\ |\{(a,c) \in \mathcal{D}^-\}| = |\{(a,c) \in \mathcal{D}^+\}|$

La première condition est une supposition sur le taux global de reussite des demandes d'aide. L'égalité du nombre de données positives et négative suggère un taux de réussite de 50%. La deuxième condition popose l'hypothèse qu'une entreprise n'a pas spécialement plus de chance qu'une autre d'obtenir une aide au sens où chacune des entreprises à autant de reussite que d'echec dans leurs choix d'aides. La troisième condition réalise l'hypothèse que le taux de réussite d'une aide est de 50% aussi.  Ces trois conditions semblent necessaires pour ne pas engendrer des données biaisées. Pour une égalité absolue, il nous faudrait introduire un peu de théorie des graphes bipartite et implémenter un algorithme non-polynomial de choix d'arêtes. Si on flexibilise un peu la condition on peut proposer un choix aléatoire avec des pondération qui permet de retrouver les deux premières conditions, et une répartition des aides a peu près similaire entre les données positives et négatives.

In [6]:
def addNonMatchingDataToDf(df, seed):
    """This function create a new Pandas DataFrame from the DataFrame df by adding non matching combinaisons
    of mt and ademe columns. It will also add a result column set to 1 for the original rows of df 
    and set to 0 for the new created rows.
    
            Parameter
            ---------
            df : A Pandas DataFrame
            A Pandas DataFrame from with we get the positives matched values.
            
            seed : An integer 
            A seed to reproduce the random choice of emulated negative data.
            
            Returns
            -------
            df_all : A Pandas DataFrame
            A Pandas Dataframe where we add to df negatives matchs and a label.
    """
    
    N = chooseNegMatchRandom(df,seed)
    
    index_ademe_list = []
    index_mt_list = []

    for m in N:
        index_ademe = df[df['nomBeneficiaire_ademe'] == m[0]].index[0]
    
        for i in m[1]:
            index_mt = df[df['id_mt'] == i].index[0]
            index_ademe_list.append(index_ademe)
            index_mt_list.append(index_mt)

    col_ademe = [c for c in df.columns if "_ademe" in c]
    col_mt = [c for c in df.columns if "_mt" in c]

    df_ademe = df[col_ademe].iloc[index_ademe_list]
    df_ademe.index = range(df.index[-1] +1, df.index[-1] + 1 + len(df_ademe))

    df_mt = df[col_mt].iloc[index_mt_list]
    df_mt.index = range(df.index[-1] +1, df.index[-1] + 1 + len(df_mt))

    df_pos = addResultColumn(df, 1)
    df_neg = df_ademe.join(df_mt)
    df_neg_result = addResultColumn(df_neg, 0)

    df_all = pd.concat([df_pos, df_neg_result])
    
    return df_all


    
    
def chooseNegMatchRandom(df, seed):
    """This function return a list of couples (company, list_of_mt_id), associating to every company of 
    df a list of id of Mission Transition aids which doesnt match with the company in the df dataset. 
    These choises are made randomly using the given seed.
    
            Parameter
            ---------
            df : A Pandas DataFrame
            A Pandas DataFrame from with we get the positives matched values.
            
            seed : An integer 
            A seed to reproduce the random choice of emulated negative data.
            
            Returns
            -------
            negative_match_list : A list of couples
            A list of couple such that every first element of the couple is a company and every second
            element of the couple is a list of id_mt of aids. Every pair (company, aid) will be negative."""
    
    random.seed(seed)
    neg_match_list = []
    
    # Series of mt aids id with frequence of apparition.
    aid_frequences = df["id_mt"].value_counts()
    
    # Series of companies with frequence of apparition.
    company_fequences = df["nomBeneficiaire_ademe"].value_counts()
    
    # For every company we choose a list of Mission Transition aids
    for company in set(df['nomBeneficiaire_ademe'].values):
        matched_aids = df[df["nomBeneficiaire_ademe"] == company]['id_mt']
        aids_frequences_without_matched = aid_frequences.drop(axis=0, index=matched_aids, inplace = False)
        
        # The number of occurence of the company into df 
        k = company_fequences[company]
        
        # We choose 
        neg_matchs_cpny = random.choices(aids_frequences_without_matched.index, 
                                         aids_frequences_without_matched.values,
                                         k=k)
        
        neg_match_list.append((company,neg_matchs_cpny))
        
    return neg_match_list




def addResultColumn(df, value):
    """This return a copy of the Pandas DataFrame df where we add a column name result,
    and where every entry is equal to 1.
            
            Parameter
            ---------
            df : A Pandas DataFrame.
                The Pandas DataFrame which we have to copy and add a result column.
            
            Return
            ------
            new_df : A Pandas DataFrame
                The new DataFrame with added column.  
    """

    df_new = df.copy()
    df_new["result"] = value
    return df_new


In [7]:
addNonMatchingDataToDf(df, 18)

,dateConvention_ademe,nomBeneficiaire_ademe,montant_ademe,pourcentageSubvention_ademe,DEPET_ademe,naf1etlib_ademe,naf2etlib_ademe,naf3etlib_ademe,naf4etlib_ademe,naf5etlib_ademe,id_mt,environmentalTopics_mt,fundingTypes_mt,subventionRateUpperBound_mt,subventionRateLowerBound_mt,applicationStartDate_mt,result
0,2021-04-12,CLEMENT SAS,6500.0,1.0,71,Industrie manufacturière,Fabrication de machines et équipements n.c.a.,Fabrication de machines de formage des métaux ...,Fabrication de machines de formage des métaux,Fabrication de machines-outils pour le travail...,44,[{'name': 'Achat & location véhicules peu poll...,['Autre aide financière'],NaN,NaN,2021-11-09T15:42:49+00:00,1
1,2021-03-25,INDUSTRIE DOLOISE DE MICRO-MECANIQUE,12150.0,1.0,39,Industrie manufacturière,"Fabrication de produits métalliques, à l'excep...",Traitement et revêtement des métaux ; usinage,Usinage,Mécanique industrielle,44,[{'name': 'Achat & location véhicules peu poll...,['Autre aide financière'],NaN,NaN,2021-11-09T15:42:49+00:00,1
2,2021-03-23,TRANSDEV BFC SUD,16000.0,1.0,71,Transports et entreposage,Transports terrestres et transport par conduites,Autres transports terrestres de voyageurs,Autres transports terrestres de voyageurs n.c.a.,Transports routiers réguliers de voyageurs,44,[{'name': 'Achat & location véhicules peu poll...,['Autre aide financière'],NaN,NaN,2021-11-09T15:42:49+00:00,1
3,2021-03-23,LOCA TRAVAUX EURL,16400.0,1.0,90,Activités de services administratifs et de sou...,Activités de location et location-bail,"Location et location-bail d'autres machines, é...",Location et location-bail de machines et équip...,Location et location-bail de machines et équip...,44,[{'name': 'Achat & location véhicules peu poll...,['Autre aide financière'],NaN,NaN,2021-11-09T15:42:49+00:00,1
4,2021-03-26,RODESCHINI SAS,21500.0,1.0,70,Construction,Travaux de construction spécialisés,Autres travaux de construction spécialisés,Autres travaux de construction spécialisés n.c.a.,Travaux de maçonnerie générale et gros œuvre d...,44,[{'name': 'Achat & location véhicules peu poll...,['Autre aide financière'],NaN,NaN,2021-11-09T15:42:49+00:00,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
943,2021-03-25,BINIC GASTRONOMIE,4225.0,1.0,22,Industrie manufacturière,Industries alimentaires,Transformation et conservation de la viande et...,Préparation de produits à base de viande,Préparation industrielle de produits à base de...,269,"[{'name': 'Energies Renouvelables', 'id': 22}]",['Subvention'],70.0,NaN,2021-11-09T15:42:47+00:00,0
944,2021-03-05,OGEC COLLEGE LE PRIEURE ET LYCEE CATHOLIQUE DE...,8040.0,1.0,41,Enseignement,Enseignement,Enseignement secondaire,Enseignement secondaire général,Enseignement secondaire général,395,"[{'name': 'Energies Renouvelables', 'id': 22}]",['Subvention'],70.0,NaN,2021-11-09T15:42:47+00:00,0
945,2018-10-09,INSTITUT THERAPEUTIQUE EDUCATIF ET PEDAGOGIQUE,2142.0,1.0,81,Santé humaine et action sociale,Hébergement médico-social et social,Hébergement médicalisé,Hébergement médicalisé,Hébergement médicalisé pour enfants handicapés,266,"[{'name': 'Energies Renouvelables', 'id': 22},...",['Subvention'],70.0,NaN,2021-11-09T15:42:47+00:00,0
946,2021-02-26,MONTPELLIER PROPRETE SERVICES,12232.0,1.0,30,Activités de services administratifs et de sou...,Services relatifs aux bâtiments et aménagement...,Activités de nettoyage,Nettoyage courant des bâtiments,Nettoyage courant des bâtiments,266,"[{'name': 'Energies Renouvelables', 'id': 22},...",['Subvention'],70.0,NaN,2021-11-09T15:42:47+00:00,0


On compare les distibutions des aides pour nos données négatives et positives.

In [8]:

def negMatchAidsDistribution(neg_match_list):
    """This function return a pandas Dataframe with aids id of the neg_match_list with 
    occurences.
    
            Parameter
            ---------
            neg_match_list : list
            A list of couple where the first element of every couple is a company, and the second 
            is a list of aid indices. 
            
            Returns
            -------
            df_aids_occ : A Pandas DataFrame
            A Pandas Dataframe where the indices are the id of aids and the values are the occurence of this aids in 
            neg_match_list.
            
            """
    L = []
    
    for m in neg_match_list :
        [L.append(i) for i in m[1]]

    df_index = pd.DataFrame(data = L)
    
    return df_index.value_counts()

In [9]:
N = chooseNegMatchRandom(df, 18)
negMatchAidsDistribution(N)

44     118
268     62
266     62
274     42
395     39
275     31
269     23
181     19
267     16
190     10
40       9
161      9
191      6
200      5
203      4
163      4
162      3
43       3
185      2
270      2
42       2
182      1
41       1
38       1
dtype: int64

In [10]:
df["id_mt"].value_counts()

44     167
268     59
266     50
395     48
274     32
275     24
181     17
269     16
267      9
203      8
190      8
161      7
40       7
162      3
191      3
185      2
163      2
41       2
182      2
42       1
180      1
43       1
38       1
199      1
200      1
270      1
46       1
Name: id_mt, dtype: int64